In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras import optimizers

import numpy as np
import json

In [2]:
batch_size  = 64
epochs      = 100
latent_dim  = 256

In [3]:
# Get training/testing data 

filepath = 'D://NLP/translation2019zh/translation2019zh_valid.json'


input_texts  = []
target_texts = []

input_characters  = set()
target_characters = set()


fd = open(filepath, 'r', encoding='utf-8')

i = 0
lines = fd.readlines()

for line in lines:
    i += 1
    if i == 2048:
        break
    
    line = line.strip('{').strip('}\n')
    
    input_text   = line.split('"english": ')[1].split(', "chinese": ')[0].strip('"')
    target_text  = line.split(', "chinese": ')[1].strip('"')
    
    #print(input_text)
    #print(target_text)
    
    target_text  = '\t' + target_text
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            
# 字典排序            
input_characters  = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

# 計算編碼器、解碼器的最大長度
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

# 以dict儲存字典單字及序號
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# 設定編碼器、解碼器input起始值(均為0矩陣)
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


# 設定 encoder_input、decoder_input對應的順序    
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.

    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

Number of samples: 2047
Number of unique input tokens: 161
Number of unique output tokens: 2940
Max sequence length for inputs: 256
Max sequence length for outputs: 175


In [4]:
# 建立 encoder LSTM 隱藏層
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# 捨棄 output，只保留記憶狀態 h 及 c
encoder_states = [state_h, state_c]

# 建立 decoder LSTM 隱藏層
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# decoder 記憶狀態不會在訓練過程使用，只會在推論(Inference)使用
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 定義模型，由 encoder_input_data 及 decoder_input_data 轉換為 decoder_target_data 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 訓練
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
          
          
# 儲存模型及結果
#model.save('s2s.h5')

Epoch 1/100
26/26 [==============================] - 96s 3s/step - loss: 1.4732 - val_loss: 1.3672
Epoch 2/100
26/26 [==============================] - 87s 3s/step - loss: 1.4253 - val_loss: 1.3616
Epoch 3/100
26/26 [==============================] - 87s 3s/step - loss: 1.4205 - val_loss: 1.3574
Epoch 4/100
26/26 [==============================] - 89s 3s/step - loss: 1.4167 - val_loss: 1.3551
Epoch 5/100
26/26 [==============================] - 100s 4s/step - loss: 1.4144 - val_loss: 1.3540
Epoch 6/100
26/26 [==============================] - 99s 4s/step - loss: 1.4123 - val_loss: 1.3529
Epoch 7/100
26/26 [==============================] - 104s 4s/step - loss: 1.4105 - val_loss: 1.3525
Epoch 8/100
26/26 [==============================] - 98s 4s/step - loss: 1.4094 - val_loss: 1.3508
Epoch 9/100
26/26 [==============================] - 97s 4s/step - loss: 1.4082 - val_loss: 1.3488
Epoch 10/100
26/26 [==============================] - 97s 4s/step - loss: 1.4069 - val_loss: 1.3498
Epoch 1

Epoch 83/100
26/26 [==============================] - 92s 4s/step - loss: 1.3948 - val_loss: 1.3499
Epoch 84/100
26/26 [==============================] - 91s 4s/step - loss: 1.3945 - val_loss: 1.3497
Epoch 85/100
26/26 [==============================] - 92s 4s/step - loss: 1.3943 - val_loss: 1.3520
Epoch 86/100
26/26 [==============================] - 93s 4s/step - loss: 1.3948 - val_loss: 1.3488
Epoch 87/100
26/26 [==============================] - 93s 4s/step - loss: 1.3944 - val_loss: 1.3496
Epoch 88/100
26/26 [==============================] - 93s 4s/step - loss: 1.3943 - val_loss: 1.3501
Epoch 89/100
26/26 [==============================] - 92s 4s/step - loss: 1.3942 - val_loss: 1.3506
Epoch 90/100
26/26 [==============================] - 92s 4s/step - loss: 1.3941 - val_loss: 1.3505
Epoch 91/100
26/26 [==============================] - 93s 4s/step - loss: 1.3938 - val_loss: 1.3512
Epoch 92/100
26/26 [==============================] - 92s 4s/step - loss: 1.3942 - val_loss: 1.3508


In [5]:
# 定義編碼器取樣模型
encoder_model = Model(encoder_inputs, encoder_states)

# 定義解碼器的input
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 定義解碼器 LSTM 模型
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
    
# 以編碼器的記憶狀態 h 及 c 為解碼器的記憶狀態  
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# 建立反向的 dict，才能透過查詢將數值轉回文字
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


# 模型預測，並取得翻譯結果(中文)    
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

# 測試100次
for seq_index in range(2):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('*')
    print('Input sentence:', input_texts[seq_index])
    try:
        print('Decoded sentence:', decoded_sentence)
    except:
        # 出現亂碼，以?取代
        print('Decoded sentence:', decoded_sentence.encode('ascii', 'replace'))
        #print("error:", sys.exc_info()[0])

*
Input sentence: Slowly and not without struggle, America began to listen.
Decoded sentence: 我我我的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
*
Input sentence: I didn't own a Thesaurus until four years ago and I use a small Webster's dictionary that I'd bought at K-Mart for 89 cents.
Decoded sentence: 我我我的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的


In [6]:
model.save('s2s.h5')